In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/resume-and-job-description/training_data.csv
/kaggle/input/resume-and-job-description/Resume.csv
/kaggle/input/ressume-tokenized/resumes_text_tokenized


# Resume Data Preprocessing

In [2]:
resumes_path = '/kaggle/input/resume-and-job-description/Resume.csv'
jobdescriptions_path = '/kaggle/input/resume-and-job-description/training_data.csv'

In [3]:
resumes_raw = pd.read_csv(resumes_path)
jobdescriptions_raw = pd.read_csv(jobdescriptions_path)

In [4]:
resumes_text = [text for text in resumes_raw['Resume_str']]

In [9]:
import spacy
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")
import regex as re

In [15]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")
resumes_text_tokenized=[]

def clean_and_tokenize(resume_text):
    stopwords = {"and", "the", "in", "of", "references", "contact", "information", '/'}
    date_pattern = r'\b\d{1,2}/\d{4}\b'
    year_pattern = r'\b\d{4}\b'
    pattern = r'ï¼\x8d'

    doc = nlp(resume_text)

    filtered_tokens = []

    for token in doc:
        token_text = token.text.lower()
        if (
            token_text not in stopwords
            and not token.is_punct
            and not token.is_space
            and len(token_text) > 1
            and not re.match(date_pattern, token_text)
            and not re.match(year_pattern, token_text)
            and not re.match(pattern, token_text)
        ):
            filtered_tokens.append(token_text)

    return filtered_tokens

for elt in tqdm(resumes_text):
    resumes_text_tokenized.append(clean_and_tokenize(elt))


100%|██████████| 2484/2484 [05:04<00:00,  8.16it/s]


In [16]:
resumes_category = [category for category in resumes_raw['Category']]

In [17]:
resumes_text_tokenized=[" ".join(elt) for elt in resumes_text_tokenized ]

# Job Data Preprocessing

In [18]:
job_text=[text for text in jobdescriptions_raw['job_description'][:15]]

In [19]:
len(job_text)

15

In [23]:
import nltk

In [24]:
def preprocess(txt):
    txt = re.sub('[^a-zA-Z]', ' ', txt)
    txt = re.sub('http\S+\s*', ' ', txt)  # remove URLs
    txt = re.sub('RT|cc', ' ', txt)  # remove RT and cc
    txt = re.sub('#\S+', '', txt)  # remove hashtags
    txt = re.sub('@\S+', '  ', txt)  # remove mentions
    txt = re.sub('\s+', ' ', txt)  # remove extra whitespace
    # tokenize word
    txt = nltk.tokenize.word_tokenize(txt)
    # remove stop words
    txt = [w for w in txt if not w in nltk.corpus.stopwords.words('english')]
    return txt

In [25]:
job_text_tokenised=[preprocess(elt) for elt in job_text]

In [26]:
job_text_tokenised=[" ".join(elt) for elt in job_text_tokenised]

In [27]:
job_text_tokenised.extend(resumes_text_tokenized)

In [31]:
data=job_text_tokenised

# Embedding using DistilBERT

In [32]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [38]:
import tensorflow_hub as hub
distilbert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
distilbert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/2")


In [39]:
def get_sentence_embeding(sentences):
    preprocessed_text = distilbert_preprocess(sentences)
    return distilbert_encoder(preprocessed_text)['pooled_output']


In [41]:
e=get_sentence_embeding(data)

In [49]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[13]])

array([[0.8329296]], dtype=float32)

In [84]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

top_resumes_per_job = []

top_n_resumes = 5

for i in range(len(job_text)):
    similarity_scores = cosine_similarity([e[i]], e[15:])
    top_resume_indices = np.argsort(similarity_scores[0])[-top_n_resumes:][::-1]
    top_resumes_indices = [(idx+15, similarity_scores[0][idx]) for idx in top_resume_indices]
    top_resumes_per_job.append(top_resumes_indices)

# 'top_resumes_per_job' now contains the top 5 resumes and their indices for each job description


In [85]:
top_resumes_per_job

[[(1229, 0.9472568),
  (2214, 0.9470049),
  (1561, 0.94594026),
  (1945, 0.94533277),
  (1578, 0.94455147)],
 [(1094, 0.97162044),
  (209, 0.9662297),
  (677, 0.96410495),
  (1678, 0.9628105),
  (231, 0.9623916)],
 [(2078, 0.97966266),
  (1307, 0.97854924),
  (2129, 0.9781345),
  (1634, 0.9769335),
  (1278, 0.9761294)],
 [(156, 0.9739831),
  (2136, 0.9722793),
  (521, 0.971405),
  (60, 0.97107005),
  (941, 0.97036916)],
 [(1947, 0.97478414),
  (60, 0.9742451),
  (618, 0.9728371),
  (149, 0.9727694),
  (1047, 0.97222006)],
 [(1471, 0.92392135),
  (1794, 0.9233513),
  (2122, 0.9228376),
  (1046, 0.9223145),
  (1512, 0.92170095)],
 [(1252, 0.97334445),
  (1021, 0.9662041),
  (221, 0.96603954),
  (1679, 0.96533906),
  (1699, 0.965058)],
 [(1734, 0.9458207),
  (762, 0.9433645),
  (2271, 0.9373971),
  (1018, 0.9371064),
  (1274, 0.9370818)],
 [(194, 0.9643845),
  (581, 0.9575571),
  (687, 0.9568101),
  (226, 0.95613456),
  (237, 0.95587367)],
 [(194, 0.9686425),
  (975, 0.96248555),
  (2328,

In [87]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[2214]])

array([[0.94700485]], dtype=float32)

In [75]:
scores=cosine_similarity([e[0]], e[15:])

# Result

In [101]:
for i in range(len(job_text)):
    print("Top 5 resumes ID's for Job description index {}".format(i))
    for j in range(5):
        print(resumes_raw.iloc[(top_resumes_per_job[0][j][0]-15),0])

Top 5 resumes ID's for Job description index 0
65062795
21297521
10549585
29087505
16449850
Top 5 resumes ID's for Job description index 1
65062795
21297521
10549585
29087505
16449850
Top 5 resumes ID's for Job description index 2
65062795
21297521
10549585
29087505
16449850
Top 5 resumes ID's for Job description index 3
65062795
21297521
10549585
29087505
16449850
Top 5 resumes ID's for Job description index 4
65062795
21297521
10549585
29087505
16449850
Top 5 resumes ID's for Job description index 5
65062795
21297521
10549585
29087505
16449850
Top 5 resumes ID's for Job description index 6
65062795
21297521
10549585
29087505
16449850
Top 5 resumes ID's for Job description index 7
65062795
21297521
10549585
29087505
16449850
Top 5 resumes ID's for Job description index 8
65062795
21297521
10549585
29087505
16449850
Top 5 resumes ID's for Job description index 9
65062795
21297521
10549585
29087505
16449850
Top 5 resumes ID's for Job description index 10
65062795
21297521
10549585
29087

In [ ]:
print(resumes_raw.loc[(top_resumes_per_job[0][j][0]-15):0])